## The Elephant in the Room
Okay, so there are about 50 billion posts about [scikit-learn](https://scikit-learn.org/stable/)(aka `sklearn`) and it's many amazing features, so what's the point of one more? This is a fair question, as many of them are truly excellent (even the internal documentation is brilliant) covering many possible topics. I personally have learned a ton through these posts and hope the amazing content continues. 

However, I feel like there is a gap that needs to be addressed. While there is so much available information about how to train and fit the latest and greatest algorithm, there is a corresponding lack of what it takes to use these methods "safely". [Machine Learning](https://en.wikipedia.org/wiki/Machine_learning) is _ultimately_ based on a strong theoretical framework that deals with the data-generating process (and associated facets of this) and how this can be exploited. The specific algorithms attempt to address some specific aspect of the framework and, in doing so, make several assumptions (e.g. "you are probably similar to your neighbor", "features are independent, given their label"). But these assumptions aren't always clear and can create some bad situations where the wrong tool is used in the wrong way/situation. What I hope to show in this post (and several others) is a way to use `sklearn` in a robust way, where some best practices can be followed.

In particular, these next posts will go through the implementation of [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) and [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV). These approaches are of great benefit in that they help the DS/ML practitioner prevent things like "[leakage](https://machinelearningmastery.com/data-leakage-machine-learning/)" or [over/underfitting](https://en.wikipedia.org/wiki/Overfitting). These topics take a while to go through, so this is likely going to be split into a few different posts.

**WARNING:** Though these approaches will help any DS/ML practitioner make better products, there is no guarantee that you are using the right algorithm in the correct way, or that you have collected/cleaned your data appropriately. What we do is as much art as it is science, and, like craftspeople from long ago, we should strive to learn as many tools and techniques as thoroughly as we can to ensure we create the best products possible. To that end, as always, I look forward to hearing from people who know more about ways that my own code/understanding could be improved. Never hesitate to reach out to me.

So without further ado, onto **Pipelines**!!

## Pipelines
The Pipeline object is one of the most useful tools within the scikit infrastructure. Though very simple and flexible to employ, they provide a powerful means to automate ML workflows which is especially useful when the models will be fit using **Cross-Validation (CV)** and/or require a number of transformations or preprocessing steps. Prior to this, most ML users would have to write their own custom functions to perform these operations. This was not only time-consuming and exhaustive (particularly when trying to make one function play nicely with subsequent functions) but also risky, as there are a number of places where inadvertent leakage could occur.  

A classic example of leakage occurred when using CV in conjunction with **scaling** the data.  Usually data was scaled first, and then CV was applied. This seems perfectly reasonable or even trivial. But doing this "taught" each individual observation something about the entire dataset which would then go into the modeling process. For instance, if MinMaxScaling were used, each datapoint is now encoded with information about the size of the largest and smallest observations in the data set, even if those values were being held out of the specific training folds.  
  

**Pipelines** make it possible to "do the right thing" even if you have a number of tasks to accomplish per training fold.  

We'll start by making some generic imports

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

### A Regression example

We are loading the "Boston Housing" dataset in this case (from sklearn).

Since the focus is more on the capabilities and less on _actually_ solving this, we won't do much in the way of EDA. In order to show off tuning multiple hyperparameters, we will use a Random Forest Regressor.

We start by just importing the dataset and then making testing and training data.  

**NOTE:** I'm leaving this call in for the "Boston Housing" dataset for now, but I do agree with the maintainers that it's a pretty messed up dataset with over racial issues.  So I'll leave the warning here as a reminder and will later correct this to use either their suggestion or something else.

In [2]:
# Loading the RF Regressor
from sklearn.ensemble import RandomForestRegressor

# Importing data; keeping it in this form to enable keeping feature names
boston = datasets.load_boston(return_X_y = False)
regression_X, regression_y = boston.data, boston.target

# Creating Test and Training sets
regression_X_train, regression_X_test, regression_y_train, regression_y_test = train_test_split(
    regression_X, regression_y, test_size = 0.2, random_state = 1234)

/usr/local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e

Now at this stage in the process, we know we will do Cross-Validation to make sure we are reducing overfitting. However, we also have not only model hyperparameters, but maybe a few other aspects we'd like to try. For instance, we may want to find out if scaling the data or performing dimension reduction will help. As discussed above, these operations need to be performed within each step of the Cross-Validation process, and below we will show how this is done within Pipelines.

In [3]:
# Creating a Scaler object
scaler = StandardScaler()
# Creating a PCA object
pca = PCA(random_state = 1234)
# Initialize the model
rf = RandomForestRegressor(random_state = 1234, n_estimators = 50,
                          max_features = 'sqrt')

# Now we will create an execution plan using Pipelines. This will provide an execution plan during GridSearchCV
#
# To do so, we create a Pipeline object and we name the steps we call in the order we want them run.  The names used 
#        are arbitrary.

pipe = Pipeline(steps = [
                    ('scaling', scaler),
                    ('pca', pca), 
                    ('random_forest', rf)
        ])

That's it -- we've just made the pipeline and could move forward.  But you will have noticed that each of the three objects called above have various parameters that could/should be called and weren't. Don't worry! These are the hyperparameters we want to test later, but this happens as part of...

## GridSearchCV
**GridSearchCV** is the other great feature of scikit; the ability to run Cross-Validation while also tuning hyperparameters. The main worker of GridSearchCV is the _param_grid_ object. With this object, you can reference named objects in the Pipeline object we made earlier, such as the number of Principal Components to use or model-specific hyperparameters.  In fact, there are even more advanced options available but we'll discuss later. But here we will try to find better values for PCA and the RF Regressor.

In [4]:
# Put in some values to explore for PCA and the RF Regressor
param_grid = {
    'pca__n_components': [4, 9, 13],
    'random_forest__max_depth': [3, 4],
    'random_forest__min_weight_fraction_leaf': [0.1, 0.2]
}

While this is a fairly simple example, it illustrates the simplicity of creating these objects.  

Note that the names we assigned when we make the _Pipeline_ object above are how we specify which object is being tested.  In this way, any parameter belonging to the method can be tested by simply calling the name, '__' and then the method name.

e.g. `'random_forest' + '__' + 'min_impurity_split' `

In [5]:
# Initialize the GridSearchCV object with 5-fold CV
# Here Verbose is set to 10 to show more information of the fitting process; by default it is zero
regression_cv = GridSearchCV(estimator = pipe, param_grid = param_grid, n_jobs = -1, cv = 5, verbose = 10)

# Now let's look at the actual object as part of a sanity check
print(regression_cv)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaling', StandardScaler()),
                                       ('pca', PCA(random_state=1234)),
                                       ('random_forest',
                                        RandomForestRegressor(max_features='sqrt',
                                                              n_estimators=50,
                                                              random_state=1234))]),
             n_jobs=-1,
             param_grid={'pca__n_components': [4, 9, 13],
                         'random_forest__max_depth': [3, 4],
                         'random_forest__min_weight_fraction_leaf': [0.1, 0.2]},
             verbose=10)


In [6]:
# Now to fit the model
#  The output is large but this is only because of the 'verbose' argument earlier;
#  setting this value to 0 will suppress nearly everything.
regression_cv.fit(regression_X_train, regression_y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaling', StandardScaler()),
                                       ('pca', PCA(random_state=1234)),
                                       ('random_forest',
                                        RandomForestRegressor(max_features='sqrt',
                                                              n_estimators=50,
                                                              random_state=1234))]),
             n_jobs=-1,
             param_grid={'pca__n_components': [4, 9, 13],
                         'random_forest__max_depth': [3, 4],
                         'random_forest__min_weight_fraction_leaf': [0.1, 0.2]},
             verbose=10)

In [7]:
# Let's see what the best fit is and the error terms:
print(regression_cv.best_params_)
print('*'*50)
print(regression_cv.best_score_)

{'pca__n_components': 4, 'random_forest__max_depth': 4, 'random_forest__min_weight_fraction_leaf': 0.1}
**************************************************
0.5476048369309383


Something very cool (but a bit overwhelming, unfortunately) is the ability to look at the raw CV outputs.

In [8]:
# This is also useful, but prints out a lot of information so it can be hard to interpret.
regression_cv.cv_results_

{'mean_fit_time': array([0.11324077, 0.1072083 , 0.10480456, 0.10521579, 0.10304217,
        0.1007061 , 0.10587311, 0.10182614, 0.10239372, 0.10151715,
        0.1006484 , 0.0768167 ]),
 'std_fit_time': array([0.00258553, 0.00363224, 0.00284345, 0.00645333, 0.00191357,
        0.00094417, 0.00289251, 0.00319431, 0.00247705, 0.0020844 ,
        0.00325542, 0.01047587]),
 'mean_score_time': array([0.01246696, 0.01047864, 0.01032453, 0.01012602, 0.00871487,
        0.00881858, 0.00903974, 0.00870447, 0.008955  , 0.00853739,
        0.00674996, 0.00418406]),
 'std_score_time': array([0.00192632, 0.00062928, 0.00066513, 0.00174225, 0.00054977,
        0.00025412, 0.00058565, 0.00031903, 0.00027355, 0.00025611,
        0.00134098, 0.00084061]),
 'param_pca__n_components': masked_array(data=[4, 4, 4, 4, 9, 9, 9, 9, 13, 13, 13, 13],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
             

Now at this point, we have created a lot of great information that we can use moving forward. But realistically, we just want to take the _best_ model learned through our CV and put that forward:

In [9]:
rf_regressor = regression_cv.best_estimator_
boston_predictions = rf_regressor.predict(regression_X_test)

from sklearn.metrics import mean_squared_error
# Printing the RMSE for fun....
print(mean_squared_error(regression_y_test, boston_predictions, squared = False))

5.986013716574084


Now at this point we would usually do a lot more to check out the tuning, fitting, etc...,.  However, this is out of the scope of this post, and we'll focus more on this in later posts.  But for now, we are only attempting to show off Pipelines and GridSearchCV. :) However, I did allude to some additional features that could be included and we'll do a Classification example to illustrate these.

### Classification Example
Again we'll use one of the built-in datasets from sklearn, but we'll expand our pipeline usage now. This dataset consists of several features we can hopefully use to correctly determine which class the wine comes from.  

To begin with, we load the data as we had before and create some test and training data.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
wine = datasets.load_wine(return_X_y = False)
classification_X, classification_y = wine.data, wine.target

# Creating Test and Training sets
classification_X_train, classification_X_test, classification_y_train, classification_y_test = train_test_split(
    classification_X, classification_y, test_size = 0.2, random_state = 1234)

Now this is where things start to get different! The pipeline object we created ensures that the same operations are followed in the same order, and that's it. In our earlier example, we initialized objects for standardization and reductions and _then_ inserted them into our Pipeline object. What's the problem with that? No matter what happens, we _had to_ perform the standardization AND do feature reduction, _even if that made our model performance suffer_!  There is good news though!  

With pipelines, we can actually initialize the `Pipeline` object with very little in it, and _then_ have parameters tested within the `param_grid`: 

In [11]:
pipe = Pipeline([('standardization', None),
                              ('reduction', None),
                              ('classifier', GradientBoostingClassifier(n_estimators = 50, 
                                                                        max_features = 'sqrt', 
                                                                        random_state = 1234))])

As you can see, we have created steps for `standardization` and `reduction` but there is actually nothing to be executed. This is actually okay, as we just created the names of steps that can be tested within the parameter grid execution. In order to do that, we make two lists that we can insert into a parameter grid. 

In [12]:
## Let's test whether to leave the data as is, or do the standard zero mean unit variance transform
standardizations = [None, StandardScaler()]

## We can also either leave the data as is, or run PCA with some other number of dimensions.
reductions = [None]
reductions.extend([PCA(x, random_state = 1234) for x in range(1, classification_X_train.shape[1] + 1, 5)] )

#So what does the reductions step actually look like:
print(reductions)

[None, PCA(n_components=1, random_state=1234), PCA(n_components=6, random_state=1234), PCA(n_components=11, random_state=1234)]


Lastly, we insert these objects into the `param_grid` object, and that's it!

In [13]:
## And wrap up all of these per stage options into a single data structure
param_grid = {
    'standardization': standardizations,
    'reduction': reductions,
    'classifier__max_depth': [1,3,5]
}

Now we insert these into the `GridSearchCV` object we did before, but we added some additional features here as well. 

Last time, we didn't specify how we wanted the models to be scored. In this case, we can call out that we want the scorer to be determined based on the model that maximizes the [`f1_weighted` score](https://stats.stackexchange.com/questions/283961/where-does-sklearns-weighted-f1-score-come-from#:~:text=The%20F1%20Scores%20are%20calculated,not%20between%20precision%20and%20recall.&text=Its%20intended%20to%20be%20used,some%20samples%20w.r.t.%20the%20others.).

In [14]:
# Initialize the GridSearchCV object with 5-fold CV and `f1_weighted` scoring.
classification_cv = GridSearchCV(estimator = pipe, param_grid = param_grid , n_jobs = -1, 
                                 cv = 5, verbose = 0, scoring = 'f1_weighted')

Now we just do what we did last time in the Regression example:

In [15]:
# Now to fit the model
classification_cv.fit(classification_X_train, classification_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardization', None),
                                       ('reduction', None),
                                       ('classifier',
                                        GradientBoostingClassifier(max_features='sqrt',
                                                                   n_estimators=50,
                                                                   random_state=1234))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [1, 3, 5],
                         'reduction': [None,
                                       PCA(n_components=1, random_state=1234),
                                       PCA(n_components=6, random_state=1234),
                                       PCA(n_components=11, random_state=1234)],
                         'standardization': [None, StandardScaler()]},
             scoring='f1_weighted')

and check out how well we did...

In [16]:
# Let's see what the best fit is and the error terms:
print(classification_cv.best_params_)
print('*'*50)
print(classification_cv.best_score_)

{'classifier__max_depth': 3, 'reduction': None, 'standardization': None}
**************************************************
0.9860006071394075


This is fascinating! This showed us that (at least in this case) the optimal model did not require any reductions or standardizations! We can now take the best estimator as we did last time and make predictions and evaluate them.

In [17]:
# Now fit the model using the best estimator...
gb_classifier = classification_cv.best_estimator_

# ...and make some predictions!
wine_predictions = gb_classifier.predict(classification_X_test)

And to get closure, we can check our results, including against our scoring metric

In [18]:
# And for fun, we can check our accuracy and f1-score
from sklearn.metrics import accuracy_score, f1_score
# Printing the accuracy....
print(accuracy_score(classification_y_test, wine_predictions))
# ...and the f1-score
print(f1_score(classification_y_test, wine_predictions, average = 'weighted'))

0.9444444444444444
0.9454861111111111


## Summary
That's it!  `Pipelines` and `GridSearchCV` gives us the ability to execute the same tasks in the same way while prohibiting snooping and other issues.  Additionally, these methods provide enough flexibility to enable us to even evaluate whether all our steps are helping the model performance.

However, there are some obvious areas for improvement that we will touch on in future posts. This will include spicing up our performance by using better CV approaches than `GridSearch`, which is computationally very intensive as your hyperparameter counts increase, as well as more advanced `Pipeline` operations as well as some other topics.  But for now, the next blog post will focus on what we should do _once we have a model we like_: **model evaluation**.

The next step in the modeling process, __model evaluation__, is going to be addressed in a future series. 

Thank you for making it to the end, and if you have any feedback, I'd love to hear it.